# Optimization Task

In [1]:
# Questions:
# - Is the penalty cost incurred every day or only when the maintenance is being done?

In [2]:
# TODO:
# - fitness function: give infeasible solutions very high fitness value -> Quintine
    # -> duplicate engines in schedule or bad but not infeasible
    # -> leftover_days < 0 -> infeasible
# - crossover -> Charlot
# - mutation -> Quintine

In [3]:
# Imports
import pandas as pd
import math
import random
import numpy as np
import time

In [4]:
np.random.seed(42)
random.seed(42)

## Global Variables

In [5]:
total_no_engines = 100
team_types = ["A", "A", "B", "B"] # List indicating which type each team has (i.e. "A" or "B")
# no_teams = 4
# a_teams = 2
# b_teams = 2

engines = list(range(1, total_no_engines + 1))

## Loading Data

In [6]:
file_path = "./data/RUL_consultancy_predictions_A3-2.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify that the data was loaded correctly
print(df.head())

# Split the "RUL;id" column into two separate columns
df[['RUL', 'id']] = df['RUL;id'].str.split(';', expand=True)

# Convert the 'RUL' column to a dictionary with 'id' column as keys
RUL = dict(zip(df['id'].astype(int), df['RUL'].astype(int)))

# print(RUL)


  RUL;id
0  135;1
1  125;2
2   63;3
3  100;4
4  103;5


## Maintenance Teams

In [7]:

maintenance_duration = {}

# Define the values for μAj
for engine in engines:
    if 1 <= engine <= 20:
        maintenance_duration[engine] = {'A': 4, 'B': 5}  # since μBj = μAj + 1
    elif 21 <= engine <= 55:
        maintenance_duration[engine] = {'A': 3, 'B': 4}  # since μBj = μAj + 1
    elif 56 <= engine <= 80:
        maintenance_duration[engine] = {'A': 2, 'B': 3}  # since μBj = μAj + 1
    else:
        maintenance_duration[engine] = {'A': 8, 'B': 9}  # since μBj = μAj + 1

# Update the values for μBj based on conditions
for engine in engines:
    if 1 <= engine <= 25:
        maintenance_duration[engine]['B'] = maintenance_duration[engine]['A'] + 1
    elif 26 <= engine <= 70:
        maintenance_duration[engine]['B'] = maintenance_duration[engine]['A'] + 2
    else:
        maintenance_duration[engine]['B'] = maintenance_duration[engine]['A'] + 1

# Print the dictionary to see the values
# for engine in engines:
#     print(f"Index engine={engine}: A={maintenance_duration[engine]['A']}, B={maintenance_duration[engine]['B']}")



## Safety due date

In [8]:
t_1 = 1
safety_due_date = {id_val: t_1 + rul - 1 for id_val, rul in RUL.items()}
# print(safety_due_date)

## Penalty costs

In [9]:
penalty_costs = {}

# Define the values for cj
for engine in range(1, 21):
    penalty_costs[engine] = 4

for engine in range(21, 31):
    penalty_costs[engine] = 3

for engine in range(31, 46):
    penalty_costs[engine] = 2

for engine in range(46, 81):
    penalty_costs[engine] = 5

for engine in range(81, 101):
    penalty_costs[engine] = 6

# Print the dictionary to see the values
# for engine, cost in penalty_costs.items():
#     print(f"Index engine={engine}: cost={cost}")


## Help functions

In [10]:
def cost(engine, day):
    """
    Calculates the incurred cost when an engine is not maintained by its safety due date.

    Parameters:
    - engine: number of the engine
    - day: current day

    Returns:
    - integer: incurred cost for the engine on the day
    """
    calculated_cost = penalty_costs[engine] * ((day - safety_due_date[engine]) ** 2)
    return min(calculated_cost, 250)

In [11]:
def update_safety_due_date(planning_horizon, engine):
    """
    Update safety due date after maintenance has been performed on the engine.
    An engine can only "fail" once during the planning period. Hence, 
    we make sure it exceeds the planning horizon.

    Parameters:
    - planning_horizon: number of days for which a maintenance schedule needs to be created
    - engine: number of the engine
    """
    safety_due_date[engine] = planning_horizon + 1

In [12]:
def complete_on_time(team, engine, start_day, planning_horizon):
    """
    Checks if the maintenance can be performed by the team on the engine within the planning period.

    Parameters:
    - team: "A" or "B" indicating the type of team
    - engine: number of the engine
    - start_day: day when the maintenance is started
    - planning_horizon: number of days for which a maintenance schedule needs to be created

    Returns:
    - boolean: whether the maintenance can be performed within the planning horizon by the team on the engine
    """
    return start_day + maintenance_duration[engine][team] - 1 <= planning_horizon    

In [13]:
def engines_requiring_maintenance(planning_horizon):
    """
    Return the engines that require maintenance within the planning period.

    Parameters:
    - planning_horizon: number of days for which a maintenance schedule needs to be created

    Returns:
    - list: engines requiring maintenance
    """
    engines_to_maintain = []
    for engine_id, due_date in safety_due_date.items():
        if due_date <= planning_horizon:
            engines_to_maintain.append(engine_id)
    return engines_to_maintain

In [14]:
def leftover_days(planning_horizon, schedule):
    """
    Calculates how many days each team still has to perform maintenance given the current schedule.

    Parameters:
    - planning_horizon: number of days for which a maintenance schedule needs to be created
    - schedule: nested list of engines maintained by each team

    Returns:
    - nested list: number of days left for each team
    """
    # Initialize a list to store the leftover time for each team
    days_left = [planning_horizon] * len(schedule)
    
    # Iterate through each team in the schedule
    for team in range(len(schedule)):
        team_type = team_types[team]
        # Iterate through each engine in the team's schedule
        for engine in schedule[team]:
            # Subtract the maintenance duration of the engine from the team's number of days left
            days_left[team] -= maintenance_duration[engine][team_type]
    
    return days_left

In [15]:
def available_teams(days_left, engine):
    """
    Returns a list of team indices that still have enough days left to perform
    maintenance on the engine.

    Parameters:
    - days_left: list containing the number of days each team has left
    - engine: number of the engine that needs to be maintained

    Returns:
    - list: numbers of the teams that are able to perform the maintenance
    """
    # Initialize a list to store indices of teams in which the maintenance still fits
    team_indices = []

    # Iterate through each team
    for team in range(len(days_left)):
        # Get the team type (i.e. "A" or "B")
        team_type = team_types[team]
        
        # Check if the team has enough time left to perform maintenance on the engine
        if days_left[team] >= maintenance_duration[engine][team_type]:
            # Add the team index to the list of fitting teams
            team_indices.append(team)
    
    return team_indices

In [16]:
def fitness(schedule):
    """
    Calculates the total incurred costs based on the maintenance schedule.

    Parameters:
    - schedule: nested list of engines maintained by each team
    
    Returns:
    - int: the total incurred costs
    """
    total_costs = 0  # Initialize total costs counter
    day = 1  # Initialize day counter

    # Iterate through each team in the schedule
    for team in range(0, len(schedule)):
        team_type = team_types[team]  # Get the type of the current team
        # Iterate through each engine maintained by the current team
        for engine in schedule[team]:
            # Add the cost of maintaining the current engine on the current day to the total costs
            total_costs += cost(engine, day)
            # Update the day counter by adding the maintenance duration of the current engine for the current team
            day += maintenance_duration[engine][team_type]

    return total_costs

In [17]:
def best_fitness_value(fitness_values):
    """
    Returns the best-found fitness value in the list of fitness values.

    Parameters:
    - fitness_values: fitness value of each individual in the population

    Returns:
    - int: index of the best-found fitness value
    - int: best-found fitness value
    """
    # Find the index of the maximum fitness value in the list
    best_index = fitness_values.index(max(fitness_values))
    
    # Return the index and value of the best-found fitness value
    return best_index, fitness_values[best_index]

In [18]:
def feasible(planning_horizon, schedule):
    """
    Determines if the schedule is feasible.
    
    Parameters:
    - planning_horizon: number of days for which a maintenance schedule needs to be created
    - schedule: nested list of engines maintained by each team

    Returns:
    - bool: whether each team can maintain their assigned engines within the planning horizon
    """
    return all(team_days_left >= 0 for team_days_left in leftover_days(planning_horizon, schedule))

In [19]:
def output_format(solution):
    """
    Transform solution to the correct output format.

    """

    return

In [20]:
def uniform_crossover(parent1, parent2):
    """
    Performs uniform crossover between two parents.

    Parameters:
    - parent1 (list): First parent representing a maintenance schedule.
    - parent2 (list): Second parent representing a maintenance schedule.

    Returns:
    - offspring1 (list): Offspring resulting from crossover with parent1.
    - offspring2 (list): Offspring resulting from crossover with parent2.
    """
    offspring1 = []
    offspring2 = []
    
    # Generate a random bit-mask
    bit_mask = [random.randint(0, 1) for _ in range(len(parent1))]

    for i in range(len(bit_mask)):
        if bit_mask[i] == 0:
            offspring1.append(parent1[i])
            offspring2.append(parent2[i])
        else:
            offspring1.append(parent2[i])
            offspring2.append(parent1[i])

    return offspring1, offspring2

## Genetic Algorithm

In [21]:
def create_random_individuals(no_individuals, planning_horizon, engines_to_maintain):
    """
    Creates a certain number of random individuals (i.e., maintenance schedules).
    Parameters:
    - no_individuals: number of random individuals that are created
    - planning_horizon: number of days for which a maintenance schedule needs to be created
    - engines_to_maintain: list of engines that require maintenance within the planning horizon

    Returns:
    - list: population containing the random individuals (i.e., maintenance schedules)
    """
    population = []
    copy_engines_to_maintain = engines_to_maintain.copy()

    # Iterate to create a certain number of individuals in the population
    for i in range(no_individuals):
        # Initialize an empty schedule for each team
        schedule = [[] for _ in range(len(team_types))]

        # Continue until all engines requiring maintenance have been assigned (as far as possible)
        while len(copy_engines_to_maintain) > 0:
            # Randomly select an engine
            engine = random.choice(copy_engines_to_maintain)
            
            # Determine how many days each team has left to perform maintenance
            days_left = leftover_days(planning_horizon, schedule)

            # Find teams that still have enough number of days left to maintain this engine
            fitting_teams = available_teams(days_left, engine)

            # If there are fitting teams, randomly select one of them and assign the engine
            if len(fitting_teams) > 0:
                team = random.choice(fitting_teams)
                schedule[team].append(engine)
                copy_engines_to_maintain.remove(engine)
        
        # Add the generated schedule to the population
        population.append(schedule)

    return population

In [22]:
def fitness_evaluation(population):
    """
    Calculates the fitness value of each individual in the population.

    Parameters:
    - population: list of individuals (i.e., maintenance schedules)

    Returns:
    - list: fitness value of each individual in the population
    """
    return [fitness(individual) for individual in population]

In [23]:
def check_termination_criterion(runs_since_best_fitness, patience):
    """
    Checks if the termination criterion has been met (i.e. ).

    Parameters:
    - runs_since_best_fitness: the number of generations the fitness value did not improve
    - patience: the number of generations the fitness value is allowed to not improve (exploration rate)
    
    Returns:
    - bool: whether the termination criteria has been met
    """
    return runs_since_best_fitness > patience

In [24]:
def select_parents(population, no_parents, fitness_values):
    """
    Selects parents using the ranking selection method.

    Parameters:
    - population: list of individuals (i.e., maintenance schedules)
    - no_parents: number of parents that are selected

    Returns:
    - list: selected parents
    """
    # Combine population and fitness values into tuples
    combined = list(zip(population, fitness_values))

    # Sort the combined list based on fitness values in ascending order (low fitness value is best)
    ranked_combined = sorted(combined, key=lambda x: x[1])

    # Extract ranked schedules from ranked tuples and assign ranks to each item in the ranked list
    ranked_schedules = [(i+1, individual) for i, (individual, _) in enumerate(ranked_combined)]
    
    probabilities = [(1 / (rank + 1), _) for (rank, _) in ranked_schedules]

    # Calculate the total sum of ranks, used to normalize the probabilities for each maintenance schedule
    total_rank_sum = sum(rank for (rank, _) in probabilities)

    # Divide the rank of each schedule by the total_rank_sum
    normalized_ranks = [(rank / total_rank_sum, individual) for (rank, individual) in probabilities]

    # Sort the normalized ranks based on the indices in the population
    probabilities = sorted(normalized_ranks, key=lambda x: population.index(x[1]))

    # Extract probabilities
    prob, _ = zip(*probabilities)

    # Select indices of individuals based on probabilities
    individuals_indices = np.random.choice(range(len(population)), size = no_parents, p = np.array(prob))

    # Return subpopulation using selected indices
    return [population[index] for index in individuals_indices]

# print(select_parents([[[1], [2], [3], [4]], [[5, 6], [7], [3], [2]]], 1, [0.9, 0.1]))

In [41]:
def crossover(parents, crossover_prob):
    """ 
    Since there are dependencies between the engines which are maintiant by each team, there will be made use of  uniform crossover since this prevents any positional biases.

    Parameters:
    - Parents: List of maintainance schedules.
    - crossover_prob: Float  representing the probability for a parent to undergo crossover.

    returns:
    list: List of offspring after crossover + parents not selected for crossover
    """

    selected_for_crossover = []
    offspring = []

    # Select parents for crossover based on crossover probability
    for parent in parents:
        random_number = random.random()
        if random_number < crossover_prob:
            selected_for_crossover.append(parent)
        else:
            offspring.append(parent) # goes to mutation phase without crossover

    print(selected_for_crossover, range(0, len(selected_for_crossover), 2))
    # Perform crossover between selected parents.
    # Make sure a parent can only be used in the crossover once.
    selection_list = list(range(0, len(selected_for_crossover), 2))
    print(selection_list)
    for i in selection_list:
        print(i)
        offspring1, offspring2 = uniform_crossover(selected_for_crossover[i], selected_for_crossover[i + 1])
        offspring.append(offspring1)
        offspring.append(offspring2)

    return offspring
    

In [42]:
parents = [
    [[1, 2, 3], [4, 5], [2,4,5], [9,3]],
    [[6, 7], [8, 9, 10], [4, 6], [1]],
    [[11, 12], [13, 14, 15], [4, 3, 2], [2, 4]],
    [[16, 17, 18], [19, 20], [2], [3, 1]]
]

# Define the crossover probability
crossover_prob = 0.9

# Call the crossover function
offspring = crossover(parents, crossover_prob)

# Display the resulting offspring
for idx, off in enumerate(offspring):
    print(f"Offspring {idx + 1}: {off}")

[[[1, 2, 3], [4, 5], [2, 4, 5], [9, 3]], [[6, 7], [8, 9, 10], [4, 6], [1]], [[11, 12], [13, 14, 15], [4, 3, 2], [2, 4]], [[16, 17, 18], [19, 20], [2], [3, 1]]] range(0, 4, 2)
[0, 2]
0
2
Offspring 1: [[1, 2, 3], [4, 5], [2, 4, 5], [1]]
Offspring 2: [[6, 7], [8, 9, 10], [4, 6], [9, 3]]
Offspring 3: [[16, 17, 18], [13, 14, 15], [4, 3, 2], [2, 4]]
Offspring 4: [[11, 12], [19, 20], [2], [3, 1]]


In [27]:
def mutation(population):
    return

In [28]:
def new_offspring():
    return

In [29]:
def genetic_algorithm(planning_horizon, population_size, no_parents, start_time, max_duration, crossover_prob):
    """
    Run the genetic algorithm.

    Parameters:
    - planning_horizon: number of days for which a maintenance schedule needs to be created
    - population_size: number of individuals in the population
    - no_parents: number of parents that are selected each iteration
    - start_time: time when the algorithm was started
    - max_duration: maximum amount of seconds the algorithm is run
    - crossover_prob: Float representing the probability for a parent to undergo crossover.

    Returns:
    - list: best-found schedule
    """
    minimum_cost = math.inf
    best_schedule = None
    patience = 50
    runs_since_best_fitness = 0

    # Find engines that require maintenance within the planning horizon
    engines_to_maintain = engines_requiring_maintenance(planning_horizon)

    # Create random individuals
    population = create_random_individuals(population_size, planning_horizon, engines_to_maintain)

    while time.time() - start_time < max_duration:
        # Evaluate fitness of the individuals
        fitness_values = fitness_evaluation(population) 

        # Get the index and value of the best-found fitness value in the current population
        current_best_index, current_min_cost = best_fitness_value(fitness_values)

        # Check if the current minimum cost is lower than the overall minimum cost
        if current_min_cost < minimum_cost:
            # Update the minimum cost and best schedule
            minimum_cost = current_min_cost
            best_schedule = population[current_best_index]
            runs_since_best_fitness = 0 # Reset the counter for runs since the a new best fitness value was found
        else:
            # Increment the counter for runs since a new best fitness value was not found
            runs_since_best_fitness += 1

        # Check if the termination criterion is met
        if check_termination_criterion(runs_since_best_fitness, patience):
            # If the termination criterion is met, return the best schedule
            return best_schedule

        # Select parents
        parents = select_parents(population, no_parents, fitness_values)

        # Perform crossover operation to generate offspring
        population = crossover(parents, crossover_prob)

        # Perform mutation operation on the offspring
        population = mutation(population)

        # Copy the best-found schedule to the new population
        population.append(best_schedule)
    
    # If the loop ends without returning, return the best schedule
    return best_schedule

In [30]:
planning_horizon = 30
population_size = 49 # Should be odd so we can easily copy the best individual to the new population
no_parents = population_size // 2 # We should be able to easily copy the best individual to the new population
max_duration = 5 * 60 # 5 minutes
crossover_prob = 0.9

# Initialize a variable to track the start time
start_time = time.time()

solution = genetic_algorithm(planning_horizon, population_size, no_parents, start_time, max_duration, crossover_prob)
print(solution)

# Structure
# maintenance_data = {
#     'Machine1': [
#         {
#             'team_type': 'A',
#             'start_day': 1,
#             'end_day': 10,
#             'cost': 1000
#         },
#         {
#             'team_type': 'B',
#             'start_day': 2,
#             'end_day': 15,
#             'cost': 1500
#         }
#     ],
#     'Machine2': [
#         {
#             'team_type': 'Team C',
#             'start_date': '2024-01-15',
#             'end_date': '2024-01-20',
#             'penalty_costs': 1200
#         }
#     ],
#     # Add more machines as needed
# }

# New structure
# schedule = [[1, 4, 5, 10], [7], [2, 8, 9], [3, 6]]
# each array corresponds to the engines each team needs to maintain
# the first two arrays are the two A teams, the last two arrays are the two B teams


[[[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []]] range(0, 21, 2)


AttributeError: 'NoneType' object has no attribute 'append'